In [1]:
import pandas as pd 

data = pd.read_csv('pnp-train.txt',delimiter='\t',encoding='latin-1', # utf8 encoding didn't work for this
                  names=['type','name']) # supply the column names for the dataframe

# this next line creates a new column with the lower-cased first word
data['first_word'] = data['name'].map(lambda x: x.lower().split()[0])

In [3]:
types = set(data['type'])
types

{'company', 'drug', 'movie', 'person', 'place'}

In [7]:
#train=data[0:16800]
#test=data[16800:]
test = data.sample(frac=0.1)
train = data[~data.index.isin(test.index)]

train.shape, test.shape

((18901, 3), (2100, 3))

we want P(T|W)

P(T|W)=P(W|T)P(T)/P(W)

In [8]:
import collections as ctr
import numpy as np

type_ctr = ctr.Counter(train.type)

def PT(T=''):
    return type_ctr[T] / len(train)
np.sum(PT(T=t) for t in list(types))

1.0

In [9]:
len(list(set(train.first_word)))

12490

In [10]:
word_ctr = ctr.Counter(train.first_word)
def PW(W=''):
    if W not in word_ctr: return 0.000001
    return word_ctr[W] / len(train)
np.sum(PW(W=w) for w in list(set(train.first_word)))

0.9999999999998136

In [11]:
counters = {}
total_counts = {}
for t in types:
    sub = train[train['type'] == t]
    counters[t] = ctr.Counter(line for line in sub['first_word'])
    total_counts[t] = sum(counters[t].values())

    # Maximum Likelihood
def PW_T(W='', T=''):
    if W not in counters[T]: return 0.000001 
    return counters[T][W]/total_counts[T]
for t in types:
    print(t, np.sum([PW_T(W=w, T=t) for w in list(set(train.first_word))]))
    

movie 1.0089810000000001
person 1.0109320000000002
drug 1.0090119999999998
place 1.009629
company 1.010928


In [12]:
# Main naive bayes classifyier P(T|W)
def PT_W(W='', T=''):
    return PW_T(W,T) * PT(T) / PW(W)

In [13]:
# Just in case
#def Ps(X, T):
#    return np.prod([Pab(X=X,W=word) for word in T])

In [14]:
PT_W(T='movie', W='fruen')

1.0

In [16]:
PT_W(T='drug', W='afrin')

1.0

In [17]:
# Get the tag with highest probability
from collections import OrderedDict
import operator
def maxProb(W=''):
    tp=list(types)
    p=[PT_W(T=t, W=W) for t in tp]
    res={}
    for i in range(len(tp)):
        res[tp[i]]=p[i]
    sorted_res = OrderedDict(sorted(res.items(), key=operator.itemgetter(1)))
    pass
    return(list(sorted_res.keys())[-1])

In [18]:
maxProb(W='fruen')

'movie'

In [19]:
maxProb(W='afrin')

'drug'

In [20]:
test['pred'] = test.first_word.map(lambda x: maxProb(x))
#train['pred'] = train.first_word.map(lambda x: maxProb(x))
from sklearn.metrics import accuracy_score
accuracy=accuracy_score(test['type'],test['pred'])
#accuracy=accuracy_score(train['type'],train['pred'])
print("Accuracy = {}".format(accuracy))

Accuracy = 0.5085714285714286


In [24]:
ctr.Counter(test.pred)

Counter({'drug': 172,
         'movie': 1532,
         'person': 255,
         'company': 116,
         'place': 25})

In [27]:
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(test['type'], test['pred'],labels=types)

TypeError: iteration over a 0-d array